In [61]:
import pandas as pd 
# Imported library

In [62]:
df=pd.read_csv(r"/Users/uk/Downloads/JOB DESCRIPTION /porter_delivery_analysis/data/raw/porter_data.csv") 
#Loaded the raw data

In [64]:
df.shape

(197428, 14)

In [45]:
df.head(5)
# Loaded first 5 rows

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,df263d996281d984952c07998dc54358,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,f0ade77b43923b38237db569b016ba25,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,f0ade77b43923b38237db569b016ba25,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,f0ade77b43923b38237db569b016ba25,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,f0ade77b43923b38237db569b016ba25,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0


In [46]:
df['created_at']=pd.to_datetime(df['created_at'], format = "%Y-%m-%d %H:%M:%S")
df['actual_delivery_time']=pd.to_datetime(df['actual_delivery_time'], format = "%Y-%m-%d %H:%M:%S")
#Converting the string to actual true datetime object

In [47]:
df['delivery_duration_minute'] = (df['actual_delivery_time'] - df['created_at']).dt.total_seconds()/60
#Calculating the delivery time in minute for each delivery(row)


In [48]:
df['delivery_duration_minute'].describe()
# Checking distrbution that if there are exremely large delivery times

count    197421.000000
mean         48.470956
std         320.493482
min           1.683333
25%          35.066667
50%          44.333333
75%          56.350000
max      141947.650000
Name: delivery_duration_minute, dtype: float64

In [49]:
df= df[(df['delivery_duration_minute']>=0) & (df['delivery_duration_minute']<=180)]
# since there were extremely large delivery time so we removed it using this block of code 

In [50]:
df['store_primary_category']=df['store_primary_category'].str.strip().str.lower()
# trims whitespaces and converts to lowercase

In [51]:
df = df[df['subtotal']>0]
df = df[df['total_items']>0]
df = df[df['max_item_price'] >= df['min_item_price']]
# removed logically invalid records

In [52]:
total_rows = len(df)
missing_info = pd.DataFrame({
    'Missing Values': df.isnull().sum(),
    'Percentage (%)': (df.isnull().sum() / total_rows * 100).round(3)
})
print(missing_info)

# Checking number of null values in each column and their % contribution to total rows

                          Missing Values  Percentage (%)
market_id                            981           0.500
created_at                             0           0.000
actual_delivery_time                   0           0.000
store_id                               0           0.000
store_primary_category              4735           2.412
order_protocol                       992           0.505
total_items                            0           0.000
subtotal                               0           0.000
num_distinct_items                     0           0.000
min_item_price                         0           0.000
max_item_price                         0           0.000
total_onshift_partners             16156           8.230
total_busy_partners                16156           8.230
total_outstanding_orders           16156           8.230
delivery_duration_minute               0           0.000


In [53]:
df=df.dropna(subset=['market_id','order_protocol'])
# we are dropping columns with missing value whose percentage contribution of total rows is less than 1%

In [54]:
df['store_primary_category'].fillna('Unknown', inplace=True)
# we are not dropping the rows with null values in store_primary_category since it contribute to 2.4% of total rows
# instead filled null places with 'Unknown'

/var/folders/85/tszsn_6n56zfd9b9p9l13r9c0000gn/T/ipykernel_98587/2701601400.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['store_primary_category'].fillna('Unknown', inplace=True)


In [55]:
cols_to_fill = ['total_onshift_partners', 'total_busy_partners', 'total_outstanding_orders']
df[cols_to_fill]=df[cols_to_fill].fillna(0)
# dropping these rows would result in significant data loss (16k rows ~ 8.2%), may reduce model accuracy, upon inspection these null are likely
# due to situations like No delivery partner were active, or temporary system failure to log operational partner data or the store was in
# an area not covered by partners at the moment, so instead of dropping we impute with 0 to preserve data integrity

In [56]:
df.isnull().sum()
# as we can see after dropping rows, filling unknown or 0, there is no columns with null value

market_id                   0
created_at                  0
actual_delivery_time        0
store_id                    0
store_primary_category      0
order_protocol              0
total_items                 0
subtotal                    0
num_distinct_items          0
min_item_price              0
max_item_price              0
total_onshift_partners      0
total_busy_partners         0
total_outstanding_orders    0
delivery_duration_minute    0
dtype: int64

In [57]:
df.duplicated().sum()
# checked if duplicates rows are present if yes  drop duplicates

np.int64(0)

In [58]:
df.drop_duplicates(inplace=True)
# dropped duplicates if there was any altough there was not

In [59]:
df.to_csv('/Users/uk/Downloads/JOB DESCRIPTION /porter_delivery_analysis/data/processed/porter_cleaned.csv',index=False)
# finally after doing operations like converting string date time to actual datetime object, calculating delivery time in minute
# for each delivery, dealing with missing value then dropping, filling as per analysis and removing duplicates our dataset is cleaned. 